In [175]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# 0. What for

# 1. Preparations
1. infer参数一是需要data，二是需要把p表示出来才能写出来loss func
2. data来自`data_selected_path`

## 1.1 全局设置


## 1.2 data 读取
1. 读取data以做SA
2. 提取出来`data_key`，以及其他计算需要的features

In [209]:
ARTIFICIAL = True
noise_pct = 0.05

# Small dataset
data_small_np_path = r'../../data/small_auctions_np.csv'

# Large data
data_large_np_path = r'E:\DATA\large_dta\large_auctions_np.csv'


if ARTIFICIAL:
    target_root_path = "../../data/artificial_targets_v2_noise=" +str(noise_pct)
else:
    target_root_path = "../../data/targets_all"

# output path: optimized parameters' saving path:
if ARTIFICIAL:
    params_opitim_delta_path_3 = "../../data/SA_PT/params_opitim_artificial_GT3_noise_"+str(noise_pct)+".csv"
else:
    params_opitim_delta_path_3 = "../../data/SA_PT/params_opitim_delta_GT3.csv"

unique_setting_NN = ['desc','bidincrement','bidfee','retail','flg_endprice']
LEN = 300

# for SA
# initial params
table_5_M = [-0.2]  # 但是0.025不是一个很好的起始值，-0.1目前最好

# lower/ upper bound
lb = [-0.3]
ub = [0.3]

import numpy as np
import csv
import os
import pandas as pd
from visdom import Visdom
from tqdm import tqdm
from GT_model.SA_modified import SABoltzmann
from sko.tools import set_run_mode
from SA_for_PT_funcs_delta_eq1 import *
import SA_for_PT_funcs_delta_eq1
from func_timeout import func_set_timeout
import func_timeout
import time

from MLP.Config.config_base import BaseConfig
config = BaseConfig()

In [210]:
data_small = pd.read_csv(data_small_np_path, encoding="utf-8")
data_large = pd.read_csv(data_large_np_path, encoding="utf-8")
data = pd.concat([data_small,data_large],axis=0,ignore_index=True)
# 由于columns不同，合并后data中肯定有nan，不过我们需要的columns不是nan就好

data_key = pd.read_csv(config.data_key_path,encoding="utf-8")

# 有`N_uniq_auction`组setting
N_uniq_auction= data_key.shape[0]
print("For PT model, there are *{}* settings waiting to be inferred.".format(N_uniq_auction))
data_key.head()

For PT model, there are *1276* settings waiting to be inferred.


,desc,bidincrement,bidfee,retail,flg_endprice
0,Sony Ericsson S500i Unlocked Mysterious Green,0.15,0.75,499.99,0
1,PSP Slim & Lite Sony Piano Black,0.15,0.75,169.99,0
2,iPod Touch Apple 8GB with Software Upgrade,0.15,0.75,299.99,0
3,Logitech Cordless Wave Keyboard and Mouse,0.15,0.75,89.99,0
4,SanDisk Cruzer Contour 4GB,0.15,0.75,59.99,0


In [211]:
all_target_file = os.listdir(target_root_path)

## 1.3 functions about 'key'

In [212]:
def select_data_fromkey(i):
    return data[(data['desc'] == data_key.iloc[i,0]) &
                (data['bidincrement'] == data_key.iloc[i,1]) &
                (data['bidfee'] == data_key.iloc[i,2]) &
                (data['retail'] == data_key.iloc[i,3]) &
                (data['flg_endprice'] == data_key.iloc[i,4])].copy()

def readin_data_fromkey(i):
    data_i_path = os.path.join(target_root_path,all_target_file[i])
    data_i = pd.read_csv(data_i_path,encoding="utf-8")
    return data_i

# 3. SA
## 3.1 define loss function
1. loss function: NLL for auctions with same `features_GT`
2. 注意只能使用理论上限max_T之内的samples

In [213]:
def loss_func(params,other_params):
    # start_time_loss = datetime.datetime.now()
    alpha = params[0]
    # delta = 1

    i = other_params[0]
    if ARTIFICIAL:
        data_i = readin_data_fromkey(i)

    else:
        data_i = readin_data_fromkey(i)

    # data_i = select_data_fromkey(i)
    # data_i.reset_index(drop=True,inplace=True)

    # Get params
    v = float(data_key.iloc[i,3])
    d = float(data_key.iloc[i,1])
    b = float(data_key.iloc[i,2])
    T_i = data_i['N'].astype(int)            # auction duration sequence

    LEN,T = get_LEN_T(v,b,d,max(T_i))

    # To get U_i given LEN and other params
    U_i = get_U_GT3(LEN, v, d, b, alpha)

    # Calculate NLL under this auction setting & PT params
    nll = get_nll_loss(T_i, U_i, LEN)

    return nll


## 3.2 do SA
1. 要对每一个setting做一次infer == 对每一个setting执行一次SA。
    - 可以并行吗？YES
2. 具体的：对每个setting `i`
    - 每一个setting `i` 对应一组samples: `data_i`
    - `data_i`中的`cnt_uniq`，也就是`A`，是相同的，表示setting `i` 进行的拍卖总次数.【但是这个`A`在计算loss的时候派不上用场】
    - `N`表示duration，因此paper公式里的$T_a$即`N[a]`
    - 因此有`A = sum(data_i['cnt_n_2'])`，其中的'cnt_n_2'表示了该行对应的`duration=N`发生的次数
    - 按照上文，求解`U[i]_t` which is a array with shape of (max(N)),也就是求解paper里的`p_t`
3.每次进行`L`次对参数的试探寻找，每次寻找对应一个温度一组新的参数。
    - 优化的完成/退出条件：温度小于`T_min`或者最低温度保持`max_stay_counter`次的不变
    - 鉴于温度小于`T_min`很难达到，因此基本上对一组参数进行优化要进行L*max_stay_counter+1次运算（loss运算）

In [214]:
params_opitim = pd.DataFrame(columns=['key_idx','alpha','delta','initial_loss','final_loss','avg_loss'])

In [215]:
# Perform SA respectively for all settings
# 设定timer，如果60s内没infer出来就跳过
@func_set_timeout(120)   # 60s
def perform_with_time(sa_boltzmann):
    return sa_boltzmann.run()

In [216]:
for i in tqdm(range(0,N_uniq_auction)):        # N_uniq_auction

    # Extract data with same `key_i` into a table
    if ARTIFICIAL:
        data_i = readin_data_fromkey(i)

    else:
        data_i = select_data_fromkey(i)
        data_i.reset_index(drop=True,inplace=True)


    T_i = data_i['N'].astype(int)          # auction duration sequence
    max_T = int(max(T_i))                  # max duration value
    A_i = len(T_i)                          # data records

    v = float(data_key.iloc[i,3])
    d = float(data_key.iloc[i,1])
    b = float(data_key.iloc[i,2])
    # d==0 suggests a fixed-price auction
    if d == 0:
        T = 1000                                   # duration limitation
        LEN = int(max(T_i))                        # U_i length
    else:
        T = np.floor((v-b)/d)                         # duration limitation
        LEN = int(T)                                  # U_i length

    initial_t = max(data['cnt_N'].astype(int))        # initial temperature
    # Calculate NLL
    print(f"> For the *{i}_th* data_key")
    print("> retail = {0},bidincrement = {1}, bidfee = {2}, infer PT's parameters".format(v,d,b))
    # print("> Initilizing SA....... \n")
    set_run_mode(loss_func, 'cached')
    set_run_mode(loss_func, 'multithreading')

    # L=30, max_stay_counter=15
    sa_boltzmann = SABoltzmann(func=loss_func, x0=table_5_M, other_params = [i], T_max=initial_t, T_min=0.9, learn_rate=0.01, L=15, max_stay_counter=5,lb=lb, ub=ub)

    print("> Now do SA....... \n")
    try:
        best_x, best_y = perform_with_time(sa_boltzmann)
    except func_timeout.exceptions.FunctionTimedOut:
        print(f'> 超时')
        # 超时，参数按照初始化的值，loss按照初始loss
        best_x, best_y = table_5_M, sa_boltzmann.generation_best_Y[0]

    print("> SA ENDS....... \n")
    print(f"> avg loss: {best_y/A_i}")

    # append the opitimized params into the df
    df_tmp = pd.DataFrame([[i,best_x[0],1,sa_boltzmann.generation_best_Y[0],best_y,best_y/A_i]],columns=['key_idx','alpha','delta','initial_loss','final_loss','avg_loss'])
    params_opitim = pd.concat([params_opitim,df_tmp],axis=0,ignore_index=True)
    # ignore_index=True could help in rearranging index

  0%|          | 0/1276 [00:00<?, ?it/s]

> For the *0_th* data_key
> retail = 499.99,bidincrement = 0.15, bidfee = 0.75, infer PT's parameters
> Now do SA....... 



  0%|          | 1/1276 [00:03<1:04:18,  3.03s/it]

STOP CODE:  Stay unchanged in the last 6 iterations
> SA ENDS....... 

> avg loss: 23.02585084794229
> For the *1_th* data_key
> retail = 169.99,bidincrement = 0.15, bidfee = 0.75, infer PT's parameters
> Now do SA....... 



  0%|          | 2/1276 [00:05<59:14,  2.79s/it]  

...

> 超时
> SA ENDS....... 

> avg loss: 139589.76606425107
> For the *1274_th* data_key
> retail = 1802.0,bidincrement = 0.01, bidfee = 0.01, infer PT's parameters
> Now do SA....... 



100%|█████████▉| 1275/1276 [9:15:33<02:05, 125.21s/it]

> 超时
> SA ENDS....... 

> avg loss: 120169.40144821124
> For the *1275_th* data_key
> retail = 1717.0,bidincrement = 0.01, bidfee = 0.01, infer PT's parameters
> Now do SA....... 



100%|██████████| 1276/1276 [9:17:38<00:00, 26.22s/it] 

> 超时
> SA ENDS....... 

> avg loss: 123695.34487104266


In [217]:
params_opitim.head()

,key_idx,alpha,delta,initial_loss,final_loss,avg_loss
0,0,-0.200000,1,7368.272271,7368.272271,23.025851
1,1,0.245135,1,6976.832807,816.373085,2.694301
2,2,0.211711,1,12433.959458,1534.758727,2.842146
3,3,0.300000,1,4943.903209,591.392737,2.337521
4,4,0.173771,1,4631.119593,1048.460645,3.039016


In [218]:
params_opitim_delta_path_3

'../../data/SA_PT/params_opitim_artificial_GT3_noise_0.05.csv'

In [219]:
# save 'params_opitim' for later check
params_opitim.to_csv(params_opitim_delta_path_3, header=True, encoding="utf-8",index=False)

# 4. append with data key
1. 根据需要更改一下，总之只执行1次
2. 添加data key以及duration等信息

In [ ]:
# params_opitim = pd.read_csv(params_opitim_delta_path_3, encoding="utf-8")
# params_opitim_withsetting = pd.concat([data_key,params_opitim],axis = 1)

In [90]:
# def to_float(x):
#     return float(x.strip('[]'))

In [91]:
# params_opitim_origin = pd.read_csv(params_opitim_delta_path_3, encoding="utf-8")
# params_opitim_origin['alpha'] = params_opitim_origin['alpha'].apply(to_float)


In [92]:
# print(params_opitim_origin['alpha'].dtype)

float64


In [93]:
# params_opitim_origin.to_csv(params_opitim_delta_path_3, header=True, index=False)